In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [5]:
!pip install sklearn

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\d.kam\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python39\\site-packages\\sklearn\\datasets\\tests\\data\\openml\\292\\api-v1-json-data-list-data_name-australian-limit-2-data_version-1-status-deactivated.json.gz'

You should consider upgrading via the 'C:\Users\d.kam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.



  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Using cached scikit_learn-0.24.2-cp39-cp39-win_amd64.whl (6.9 MB)
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.


In [2]:
data = pd.read_csv('winemag.csv', index_col=0, na_filter=False)
data.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,,Sicily & Sardinia,Etna,,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,,,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,,Alexander Peartree,,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


Из всего многообразия признаков нас будет интересовать **description** -- признак содержащий описание вина и **points** -- присужденный балл

Выделим из данных X и y

In [3]:
X = data['description']
y = data['points']

## Извлечение признаков из текстов

Перед тем как работать с текстом, его необходимо токенизировать - разбить на отдельные токены. В качестве токенов могут выступать слова, фразы, предложений и т.п. Токенизировать текст можно  помощью регулярных выражений или готовых токенизаторов. 

После токенизации нужно привести текст к нормальной форме. Речь идет о [стемминге и/или лемматизации](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html) - это схожие процессы, используемые для обработки словоформ.

Для работы лемматизации английского текста можно воспользоваться библиотекой nltk:

In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alexandermarkov/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexandermarkov/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Токенизируйте тексты. Сейчас Х - это набор из строк-текстов. Разбейте каждый текст на слова (токенизируйте его), каждое из которых написано в нижнем регистре.

(Вам могут понадобиться функция word_tokenize из nltk.tokenize, а также методы строк isalpha и lower)

In [5]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm_notebook

tokens = [list(filter(str.isalpha, word_tokenize(text.lower()))) for text in tqdm_notebook(X)]

In [6]:
tokens[0]

['aromas',
 'include',
 'tropical',
 'fruit',
 'broom',
 'brimstone',
 'and',
 'dried',
 'herb',
 'the',
 'palate',
 'is',
 'overly',
 'expressive',
 'offering',
 'unripened',
 'apple',
 'citrus',
 'and',
 'dried',
 'sage',
 'alongside',
 'brisk',
 'acidity']

Теперь, когда для каждого текста у вас есть набор слов, встречающихся в нем, можно переходить к лемматизации.
Лемматизируйте все слова с помощью WordNetLemmatizer и объедините получившиеся слова в одну строку через пробел, запишите результат в X

In [7]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

X = [' '.join([lemmatizer.lemmatize(word) for word in array]) for array in tqdm_notebook(tokens)]

In [8]:
X[0]

'aroma include tropical fruit broom brimstone and dried herb the palate is overly expressive offering unripened apple citrus and dried sage alongside brisk acidity'

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

### Bag of Words

Cоздаем вектор длиной в словарь, для каждого слова считаем количество вхождений в текст и подставляем это число на соответствующую позицию в векторе.

Построим модель BOW с помощью [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X_train)

vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d' % len(vocabulary))

description_count = vectorizer.transform(X_train)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: %s'%'; '.join(top_tokens))

Размер словаря: 26552
Top-10 слов: and; the; of; with; this; it; is; wine; flavor; in


Видно, что большая часть из топ-10 слов является не информативными - стоп-словами. Что бы они не участвовали в представление, в конструктор CountVectorizer в качестве параметра можно передать список стоп-слов:

In [11]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexandermarkov/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
stop_words = set(stopwords.words('english'))
vectorizer = CountVectorizer(stop_words=stop_words)
vectorizer.fit(X_train)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d' % len(vocabulary))

description_count = vectorizer.transform(X_train)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: %s'%'; '.join(top_tokens))

Размер словаря: 26429
Top-10 слов: wine; flavor; fruit; aroma; finish; palate; acidity; tannin; cherry; drink


Чтобы сжать векторное представление, можно "отбросить" редкие слова:

In [13]:
vectorizer = CountVectorizer(stop_words=stop_words, min_df=10).fit(X)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d'%len(vocabulary))

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

Размер словаря: 7669


### Naive Bayes

In [24]:
from sklearn.naive_bayes import MultinomialNB

In [26]:
y.unique()

array([ 87,  86,  85,  88,  92,  91,  90,  89,  83,  82,  81,  80, 100,
        98,  97,  96,  95,  93,  94,  84,  99])

In [31]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred = nb.predict_proba(X_test)

In [30]:
np.unique(y, return_counts=True)[1] / np.unique(y, return_counts=True)[1].sum()

array([0.00305453, 0.00532426, 0.01412623, 0.02327442, 0.04985728,
       0.07332405, 0.0969447 , 0.13028291, 0.13239107, 0.09406714,
       0.11856491, 0.08739642, 0.07396265, 0.04992652, 0.02891414,
       0.01181033, 0.00402397, 0.00176193, 0.00059244, 0.0002539 ,
       0.00014619])

In [44]:
def top_k_accuracy(y_true, y_prob, k):
    top_k = np.argsort(y_prob, axis=1)[:, -k:]
    predictions = nb.classes_[top_k]
    return np.any(predictions == y_true.reshape((y_true.shape[0], 1)), axis=1).mean()

In [ ]:
y_test = y_test.values

In [52]:
top_k_accuracy(y_test, y_pred, k=5)

0.7936605631635636

In [42]:
#y_test = y_test.values
np.any(nb.classes_[np.argsort(y_pred, axis=1)[:, -5:]] == y_test.reshape((y_test.shape[0], 1)), axis=1).mean()

0.7936605631635636